#Objectives
#####Web scrap Falcon 9 launch records with BeautifulSoup:


*   Extract a Falcon 9 launch records HTML table from Wikipedia

*   Parse the table and convert it into a Pandas data frame




In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [4]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]
def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out
def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass
def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    if row.p:
        thep = row.p.extract().contents
        lis = row.contents + thep
        colunm_name = ' '.join(lis)
    else:    
        colunm_name = ' '.join(row.contents)
    print(colunm_name)
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name 

In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

##TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [6]:
theobject = requests.get(static_url)
BeautyObject = BeautifulSoup(theobject.text)
BeautyObject.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

##TASK 2: Extract all column/variable names from the HTML table header

In [7]:
html_tables = BeautyObject.find_all('table')
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [8]:
column_names = []
ths = first_launch_table.find_all('th')
for each in ths:
    name = extract_column_from_header(each)
    if (name) and (len(name) > 0):
        column_names.append(name)

Flight No.

Date and time ( )

  

Launch site

Payload 

Payload mass

Orbit

Customer

Launch outcome



1

2

3

4

5

6

7



In [9]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


##TASK 3: Create a data frame by parsing the launch HTML tables

In [10]:
launch_dict= dict.fromkeys(column_names)
del launch_dict['Date and time ( )']
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [11]:
extracted_row = 0
for table_number,table in enumerate(BeautyObject.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            time = datatimelist[1]
            launch_dict['Time'].append(time)         
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)          
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)           
            payload = row[3].a.string          
            launch_dict['Payload'].append(payload)           
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)                      
            orbit = row[5].a.string          
            launch_dict['Orbit'].append(orbit)          
            try:
                customer = row[6].a.string
            except:
                customer = row[6].string
            launch_dict['Customer'].append(customer)         
            launch_outcome = list(row[7].strings)[0]          
            launch_dict['Launch outcome'].append(launch_outcome)            
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [12]:
print(len(launch_dict['Orbit']))

121
